# RDDのサンプリング

[Introduction to Spark with Python, by Jose A. Dianes](https://github.com/jadianes/spark-py-notebooks)

これまでは、マップやフィルタなどの基本的な変換と、カウント、テイク、および収集などのいくつかのアクションとともにRDDの作成を導入しました。


このノートブックでは、RDDをサンプリングする方法を説明します。変換については、多くの統計的学習シナリオで役立つため、サンプルが導入されます。次に、結果をtakeSampleアクションと比較します。

## データの取得とRDDの作成

このケースでは、KDD Cup 1999のために提供された完全なデータセットを使用します。このデータセットには、約50万件のネットワークインタラクションが含まれています。このファイルは、ローカルでダウンロードするGzipファイルとして提供されています。

In [3]:
import urllib
f = urllib.request.urlretrieve ("http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data.gz", "kddcup.data.gz")

HTTPError: HTTP Error 407: Proxy Authentication Required

これでRDDを作成することができます。

In [6]:
data_file = "./kddcup.data.gz"
raw_data = sc.textFile(data_file)

## RDDのサンプリング

Sparkでは、変換サンプルとアクションtakeSampleの2つのサンプリング操作があります。 変換を使用することで、特定のRDDのサンプルに対して連続変換を適用するようにSparkに指示できます。 アクションを使用することで、与えられたサンプルを取得し、他の標準ライブラリ（Scikit-learnなど）で使用するためにローカルメモリに保存することができます。

### サンプル変換

サンプル変換には、最大3つのパラメータが必要です。まず、サンプリングが置換で行われるかどうかです。 2番目は、サンプルサイズを小数として表したものです。最後に、ランダムにシードを提供することもできます。<br>
sample(withReplacement, fraction, seed) 指定された乱数ジェネレータシードを使用して、置換の有無にかかわらずデータの小数部分をサンプリングします。

In [7]:
raw_data_sample = raw_data.sample(False, 0.1, 1234)
sample_size = raw_data_sample.count()
total_size = raw_data.count()
print("Sample size is {} of {}".format(sample_size, total_size))
raw_data_sample.take(5)

Sample size is 489957 of 4898431


['0,tcp,http,SF,215,45076,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,239,1295,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,7,7,0.00,0.00,0.00,0.00,1.00,0.00,0.00,8,8,1.00,0.00,0.12,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,185,9020,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,2,0.00,0.00,0.00,0.00,1.00,0.00,0.00,11,11,1.00,0.00,0.09,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,233,2032,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,4,0.00,0.00,0.00,0.00,1.00,0.00,0.00,15,15,1.00,0.00,0.07,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,239,486,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.00,0.00,0.00,0.00,1.00,0.00,0.00,26,26,1.00,0.00,0.04,0.00,0.00,0.00,0.00,0.00,normal.']

しかし、変換としてのサンプリングのパワーは、一連の追加の変換の一部としてそれを行うことから来ています。 これは、集計やキーと値のペアの操作を開始するとより強力になり、Sparkの機械学習ライブラリMLlibを使用する場合に特に便利になります。

その間に、正常の割合の近似値を求めたいと考えています。 データセット内の相互作用。 以前のノートブックで行ったように、タグの総数を数えることでこれを行うことができました。 しかし、私たちはより迅速な応答を望んでおり、正確な答えは必要なく、近似値だけです。 私たちは以下のようにそれを行うことができます。

In [4]:
from time import time

# transformations to be applied
raw_data_sample_items = raw_data_sample.map(lambda x: x.split(","))
sample_normal_tags = raw_data_sample_items.filter(lambda x: "normal." in x)

# actions + time
t0 = time()
sample_normal_tags_count = sample_normal_tags.count()
tt = time() - t0

sample_normal_ratio = sample_normal_tags_count / float(sample_size)
print("The ratio of 'normal' interactions is {}".format(round(sample_normal_ratio,3)) )
print("Count done in {} seconds".format(round(tt,3)))

The ratio of 'normal' interactions is 0.199
Count done in 26.828 seconds


これをサンプリングなしの比率計算と比較してみましょう。

In [5]:
# transformations to be applied
raw_data_items = raw_data.map(lambda x: x.split(","))
normal_tags = raw_data_items.filter(lambda x: "normal." in x)

# actions + time
t0 = time()
normal_tags_count = normal_tags.count()
tt = time() - t0

normal_ratio = normal_tags_count / float(total_size)
print("The ratio of 'normal' interactions is {}".format(round(normal_ratio,3))) 
print("Count done in {} seconds".format(round(tt,3)))

The ratio of 'normal' interactions is 0.199
Count done in 50.315 seconds


我々は時間の利益を見ることができます。サンプリング後に適用する変換が多いほど、この利点は大きくなります。これは、サンプリングなしでは、すべての変換がデータの完全なセットに適用されるためです


### takeSampleアクション

私たちが必要とするのは、RDDからローカルメモリに生データのサンプルを集めて、他の非Sparkライブラリで使用するためにtakeSampleを使用することです。

構文は非常に似ていますが、この場合、サンプルサイズではなくアイテムの数を完全なデータサイズの一部として指定します。

In [6]:
t0 = time()
raw_data_sample = raw_data.takeSample(False, 400000, 1234)
normal_data_sample = [x.split(",") for x in raw_data_sample if "normal." in x]
tt = time() - t0

normal_sample_size = len(normal_data_sample)

normal_ratio = normal_sample_size / 400000.0
print("The ratio of 'normal' interactions is {}".format(normal_ratio))
print("Count done in {} seconds".format(round(tt,3)))

The ratio of 'normal' interactions is 0.1988025
Count done in 76.166 seconds


このプロセスはこれまでと非常によく似ていました。 データの約10％のサンプルを取得し、フィルタリングと分割を行いました。
しかし、わずかに小さいサンプルでも、時間がかかりました。 その理由は、Sparkがサンプリングプロセスの実行をただ分散したからです。 結果のフィルタリングと分割は、1つのノードでローカルに実行されました。